# Day 15

## Part 1

We need to explore a map and compute a shortest path.

I finally moved the `Computer` into a module.
I also do a `ComputerAsync`, which is the last one we used, and a `Computer` which use an object which methods `read_input()` and `write_output()`.

In [1]:
from intcode import Computer

In [2]:
from collections import defaultdict
from dataclasses import dataclass
from enum import Enum
from datetime import datetime


@dataclass
class Point:
    x: int
    y: int
        
    def __add__(self, o):
        return Point(self.x + o.x, self.y + o.y)
    
    def __hash__(self):
        return hash((self.x, self.y))

    
UP = Point(0, 1)
RIGHT = Point(1, 0)
DOWN = Point(0, -1)
LEFT = Point(-1, 0)

class Block(Enum):
    NOT_VISITED = -1
    WALL = 0
    EMPTY = 1
    OXYGEN = 2
    
    def __gt__(self, o):
        return self.value > o.value
    

class Robot:
    def __init__(self):
        self.map_ = defaultdict(lambda: Block.NOT_VISITED)
        self.last_map = defaultdict(lambda: datetime(1970, 1, 1))
        self.position = Point(0,0)
        self.direction = Point(0, 0)
        
        self.movements = ((DOWN, 2), (RIGHT, 4), (UP, 1), (LEFT, 3))
    
        self.map_[self.position] = Block.EMPTY
        self.last_map[self.position] = datetime.now()
        
    def input(self):
        move, i = sorted(
            ((d, i) for d, i in self.movements),
            key=lambda x: (
                self.map_[self.position + x[0]] == Block.WALL,
                self.last_map[self.position + x[0]],
            ),
        )[0]

        self.direction = move
        return i
    
    def output(self, value):
        value = Block(value)

        self.map_[self.position + self.direction] = value
        self.last_map[self.position + self.direction] = datetime.now()

        if value != Block.WALL:
            self.position += self.direction
        
    def draw(self):
        for y in range(max(p.y for p in self.map_), min(p.y for p in self.map_) - 1, -1):
            for x in range(min(p.x for p in self.map_), max(p.x for p in self.map_) + 1):
                if (x, y) == (0, 0):
                    print("O ", end="")
                    continue

                print(
                    {
                        Block.WALL: "#",
                        Block.EMPTY: ".",
                        Block.NOT_VISITED: "?",
                        Block.OXYGEN: "X",
                    }[self.map_[Point(x, y)]] + " ",
                    end=""
                )
            
            print()

Let's try our robot with a simple map:
```
# # # # #
# # . # #
# . . . #
# . # . #
# # # # #
```

In [3]:
robot = Robot()
robot.input()

2

In [4]:
robot.output(0)
robot.input()

4

In [5]:
robot.output(1)
robot.input()

2

In [6]:
robot.output(1)
robot.input()

2

In [7]:
robot.output(0)
robot.input()

4

In [8]:
robot.output(0)
robot.input()

1

In [9]:
robot.output(1)
robot.input()

4

In [10]:
robot.output(0)
robot.input()

1

In [11]:
robot.output(0)
robot.input()

3

In [12]:
robot.output(1)
robot.input()

1

In [13]:
robot.output(1)
robot.input()

1

In [14]:
robot.output(0)
robot.input()

3

In [15]:
robot.output(0)
robot.input()

2

In [16]:
robot.output(1)
robot.input()

3

In [17]:
robot.output(1)
robot.input()

2

In [18]:
robot.output(1)
robot.input()

2

In [19]:
robot.output(0)
robot.input()

3

In [20]:
robot.output(0)
robot.input()

1

In [21]:
robot.output(1)
robot.input()

3

In [22]:
robot.output(0)
robot.input()

4

In [23]:
robot.draw()

? ? # ? ? 
? # . # ? 
# . O . # 
# . # . # 
? # ? # ? 


In [24]:
program = [3,1033,1008,1033,1,1032,1005,1032,31,1008,1033,2,1032,1005,1032,58,1008,1033,3,1032,1005,1032,81,1008,1033,4,1032,1005,1032,104,99,1002,1034,1,1039,102,1,1036,1041,1001,1035,-1,1040,1008,1038,0,1043,102,-1,1043,1032,1,1037,1032,1042,1106,0,124,102,1,1034,1039,101,0,1036,1041,1001,1035,1,1040,1008,1038,0,1043,1,1037,1038,1042,1106,0,124,1001,1034,-1,1039,1008,1036,0,1041,1001,1035,0,1040,1002,1038,1,1043,101,0,1037,1042,1106,0,124,1001,1034,1,1039,1008,1036,0,1041,102,1,1035,1040,101,0,1038,1043,1001,1037,0,1042,1006,1039,217,1006,1040,217,1008,1039,40,1032,1005,1032,217,1008,1040,40,1032,1005,1032,217,1008,1039,7,1032,1006,1032,165,1008,1040,37,1032,1006,1032,165,1102,1,2,1044,1106,0,224,2,1041,1043,1032,1006,1032,179,1102,1,1,1044,1106,0,224,1,1041,1043,1032,1006,1032,217,1,1042,1043,1032,1001,1032,-1,1032,1002,1032,39,1032,1,1032,1039,1032,101,-1,1032,1032,101,252,1032,211,1007,0,39,1044,1105,1,224,1101,0,0,1044,1105,1,224,1006,1044,247,102,1,1039,1034,102,1,1040,1035,101,0,1041,1036,102,1,1043,1038,102,1,1042,1037,4,1044,1106,0,0,35,37,2,26,91,30,85,34,87,18,47,29,50,23,7,46,94,2,26,42,36,23,3,32,65,21,63,18,54,31,52,75,4,35,24,24,74,33,81,89,75,50,36,43,7,20,45,9,23,10,70,12,81,62,12,51,3,5,96,7,93,90,12,41,5,52,30,91,12,62,34,44,92,68,9,81,9,6,30,38,63,27,51,3,44,47,27,86,41,1,73,78,15,34,98,9,63,66,21,89,96,5,9,36,21,97,6,26,75,14,86,16,82,21,23,91,25,15,66,33,2,50,26,18,61,73,17,49,15,99,19,68,96,33,23,12,81,11,51,19,30,56,74,27,40,76,15,49,11,24,50,27,50,36,77,36,16,22,80,86,11,85,20,87,24,26,6,64,35,27,65,32,86,42,99,30,78,68,24,67,82,4,76,63,36,4,46,21,72,68,17,21,69,71,36,82,22,57,1,29,95,59,18,48,40,91,7,44,22,64,5,52,20,20,86,34,9,67,74,22,13,31,97,23,19,78,19,12,80,19,82,83,11,26,5,10,74,2,42,5,94,26,79,51,33,15,47,9,12,84,20,37,85,63,27,92,16,10,82,64,15,50,75,12,68,51,37,87,10,51,18,11,13,99,97,30,33,48,2,45,29,22,45,20,49,14,78,33,41,89,4,67,21,40,42,20,4,34,64,98,32,77,28,79,9,51,91,58,19,45,56,4,10,3,52,47,65,11,21,53,25,57,78,33,16,70,88,34,56,37,86,30,4,84,91,86,90,37,37,25,59,2,96,25,19,69,6,11,67,83,38,8,49,18,17,21,56,20,43,89,8,78,30,80,52,29,9,65,1,1,65,27,84,23,8,33,99,71,28,38,45,14,40,31,45,44,12,94,12,65,23,96,5,93,50,35,84,10,34,81,2,51,15,11,92,69,20,65,27,68,86,76,36,49,38,79,92,38,72,8,32,80,29,41,7,15,78,38,5,10,61,24,44,38,19,80,9,60,95,95,33,48,13,51,32,57,84,97,1,51,36,6,51,96,16,62,32,13,93,4,79,40,2,68,74,38,4,30,82,17,67,51,68,29,3,85,13,5,2,30,71,36,77,35,78,23,87,22,7,78,5,60,2,11,42,15,68,89,66,93,31,38,31,81,8,65,22,7,27,83,59,21,12,73,64,72,40,38,59,20,29,92,20,7,65,16,86,81,12,44,77,97,30,19,49,61,24,29,24,31,87,89,31,42,80,17,91,23,18,91,10,53,5,17,53,30,96,96,34,83,34,18,68,79,97,18,4,56,37,33,62,31,79,99,32,14,99,87,83,53,34,26,17,70,59,31,12,42,91,32,93,5,54,8,10,83,20,58,92,30,71,24,34,60,3,9,64,72,12,70,14,22,69,38,27,77,31,84,8,54,44,58,9,30,95,22,12,61,95,21,81,71,5,64,44,7,71,4,17,41,2,89,16,20,93,88,20,31,45,28,49,91,15,72,43,6,21,82,15,25,99,8,11,34,18,93,50,15,15,98,27,34,44,38,15,29,79,42,14,86,68,56,7,3,97,21,58,11,33,67,6,53,23,71,16,58,74,17,92,17,14,98,23,35,60,32,70,54,1,82,2,41,32,68,91,27,80,6,25,55,93,23,52,91,3,95,44,3,42,70,23,16,54,36,36,59,5,63,27,40,11,73,34,48,29,73,36,74,77,58,25,55,25,45,7,58,53,49,8,95,13,84,23,58,37,42,6,70,36,58,73,55,14,51,5,99,95,61,20,65,0,0,21,21,1,10,1,0,0,0,0,0,0]
robot = Robot()
computer = Computer(program, robot)
computer.compute()

KeyboardInterrupt: 

In [25]:
robot.draw()

? # # # # # # # # # # # # # # # ? # # # ? # # # # # # # # # # # ? # # # # # # # ? 
# . . . . . . . . . . . . . . . # . . . # . . . . . . . . . . . # . . . . . . . # 
# . # . # # # # # # # # # # # . # # # . # . # . # # # # # # # # # . # . # # # # ? 
# . # . . . . . # . . . # . # . . . . . # . # . . . . . . . . . . . # . . . . . # 
? # # # # # # . # . # . # . # # # # # . # . # # # # # # # # # # # # # # # # # . # 
# . . . . . . . # . # . . . # . . . . . # . # . . . . . # . . . . . . . . . # . # 
# . # # # # # # # . # # # # # . # # # # # . # # # . # . # . # # # # # # # . # . # 
# . # . . . . . # . . . . . . . # . . . # . # . . . # . # . . . . . # . # . # . # 
# . # . # # # . # # # # # # # # # . # . # . # . # # # . # # # # # . # . # . # . # 
# . . . # . # . # . . . # . . . . . # . . . # . # . . . # . . . . . # . . . # . # 
# . # # # . # . # . # . # . # # # # # # # # # . # # # # # . # # # # # . # # # . # 
# . . . . . # . . . # . . . . . . . . . # . . . # . . . # . . . . . # . # . . . # 
? # 

The map is *short* enough so counting by hand is quicker than writing code.
The path to the oxygen system (`X`) is 214 steps long.

## Part 2

We want to know how many time the oxygen will take to fill all the map, knowing it takes 1 minute to go one step in each directions.

In [26]:
oxygen_position = next(filter(lambda p: robot.map_[p] == Block.OXYGEN, robot.map_.keys()))
oxygen_position

Point(x=-14, y=-16)

In [27]:
map_ = robot.map_.copy()
def fill_oxygen(map_, position, count=0):
    directions = (UP, LEFT, DOWN, RIGHT)
    map_[position] = Block.OXYGEN
    
    counts = [count]
    for direction in directions:
        if map_[position + direction] == Block.EMPTY:
            counts.append(fill_oxygen(map_, position + direction, count + 1))
            
    return max(counts)

In [28]:
fill_oxygen(map_, oxygen_position)

344